In [127]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_theme()

import torch 
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.set_default_dtype(torch.float64)

%matplotlib inline

In [128]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [129]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cpu device


In [154]:
from bernstein_torch import bernstein_coeff_order10_new

# Generating P matrix
t_fin = 6.0
num = 60
tot_time = torch.linspace(0, t_fin, num)
tot_time_copy = tot_time.reshape(num, 1)
P, Pdot, Pddot = bernstein_coeff_order10_new(10, tot_time_copy[0], tot_time_copy[-1], tot_time_copy)
print(P.shape, Pdot.shape, Pddot.shape)

torch.Size([60, 11]) torch.Size([60, 11]) torch.Size([60, 11])


In [155]:
# Custom Dataset Loader 
class TrajDataset(Dataset):
    """Expert Trajectory Dataset."""
    def __init__(self, data):
        
        # Inputs
        self.inp = data[:, 0:55]
        
        # Outputs
        self.out = data[:, 55:]
        
    def __len__(self):
        return len(self.inp)
    
    def __getitem__(self, idx):
        
        # Inputs
        inp = self.inp[idx]
        
        # Outputs
        out = self.out[idx]
                 
        return torch.tensor(inp).double(), torch.tensor(out).double()

def rotate(gt_x, gt_y,theta):
    gt_x_x = [ (gt_x[k] * np.cos(theta) - gt_y[k] * np.sin(theta))  for k in range(len(gt_x))]
    gt_y_y = [ (gt_x[k] * np.sin(theta) + gt_y[k] * np.cos(theta))  for k in range(len(gt_x))]
    gt_x = gt_x_x
    gt_y = gt_y_y
    return gt_x, gt_y
    
# Load the dataset
# train_data = np.load("../datasets/toy/train_data.npy", mmap_mode="c")
DATASET_PATH =  "/mnt/e/datasets/argoverse/parsed"
import os
files = os.listdir(DATASET_PATH)

# Custom Dataset Loader 
class TrajDataset(Dataset):
    """Expert Trajectory Dataset."""
    def __init__(self, dataset_path):
        
        self.dataset_path = dataset_path
        self.files = os.listdir(DATASET_PATH)
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        arr = np.load(os.path.join(self.dataset_path, self.files[idx], f"{self.files[idx]}.npy"), allow_pickle=True)
        # map_info = np.load(os.path.join(self.dataset_path, self.files[idx], f"lanes_{self.files[idx]}.npy"), allow_pickle=True)
        agent, obstacle_tracks = None, []
        # [object_state.position[0], object_state.position[1], object_state.heading, object_state.velocity[0], object_state.velocity[1], timestamps_ns[ind], tck[track.category], vmp[track.object_type]
        # 5s of input of agent and 4 obstacles, 5s of output
        inp = np.zeros((200))
        out = np.zeros((120))
        for tracks in arr:
            if tracks[0][-2] == 3:
                agent = tracks
                last_obs = np.array(tracks)[-1]
                last_obs_x = last_obs[0]
                last_obs_y = last_obs[1]
                last_obs_vel_x = last_obs[3]
                last_obs_vel_y = last_obs[4]
                for i in range(np.array(tracks).shape[0], 110):
                    timestep = (i - np.array(tracks).shape[0] + 1)
                    tracks.append([last_obs_x + last_obs_vel_x * timestep, last_obs_y + last_obs_vel_y * timestep, last_obs[2], last_obs_vel_x, last_obs_vel_y, i, last_obs[-2], last_obs[-1]])
                agx = np.array(tracks)[10:50, 0] # position-x
                agy = np.array(tracks)[10:50, 1] # position-y
                offsetx = agx[-1]
                offsety = agy[-1]
                theta = np.arctan2(agy[-1] - agy[-2], agx[-1] - agx[-2])
                ox = np.array(tracks)[50:, 0] # position-x
                oy = np.array(tracks)[50:, 1] # position-x
                agx, agy = rotate(agx - offsetx, agy - offsety, -theta)
                ox, oy = rotate(ox - offsetx, oy - offsety, -theta)
                vx = np.array(tracks)[10:50, 3] # velocity-x
                vy = np.array(tracks)[10:50, 4] # velocity-y
                vx, vy = rotate(vx, vy, -theta)
                inp[0::5] = agx
                inp[1::5] = agy
                inp[2::5] = vx
                inp[3::5] = vy
                inp[4::5] = np.array(tracks)[10:50, 2] - theta # heading
                out[:60] = ox
                out[60:] = oy
                traj_inp = np.hstack((agx, agy)).flatten()
                traj_out = np.hstack((ox, oy)).flatten()
                b_inp = np.array([agx[-1], agy[-1], vx[-1], vy[-1], 0, 0])
                
        return torch.tensor(inp).double(), torch.tensor(out).double(), torch.tensor(traj_inp).double(), torch.tensor(traj_out).double(), torch.tensor(b_inp).double()

train_dataset = TrajDataset(DATASET_PATH)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
for _, data in enumerate(train_loader):
    inp, out, past, future, inputs = data
    print(inp.shape, out.shape)
    break
    pass
# Using PyTorch Dataloader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)

torch.Size([64, 200]) torch.Size([64, 120])


In [185]:
from beta_cvae_aug_ddn import Encoder, Decoder, Beta_cVAE, BatchOpt_DDN, DeclarativeLayer
from gru_cvae_aug_ddn import GRU_cVAE, DecoderGRU, EncoderGRU

# DDN
num_batch = train_loader.batch_size
node = BatchOpt_DDN(P, Pdot, Pddot, num_batch)
opt_layer = DeclarativeLayer(node)

# Beta-cVAE Inputs
enc_inp_dim = 200
enc_out_dim = 120
dec_inp_dim = enc_inp_dim
dec_out_dim = 8
hidden_dim = 1024 * 2
z_dim = 2

inp_mean, inp_std = 5.1077423, 20.914295

encoder = Encoder(enc_inp_dim, enc_out_dim, hidden_dim, z_dim)
decoder = Decoder(dec_inp_dim, dec_out_dim, hidden_dim, z_dim)
model = Beta_cVAE(encoder, decoder, opt_layer, inp_mean, inp_std).to(device)

encoder = EncoderGRU(enc_inp_dim, enc_out_dim, hidden_dim, z_dim, batch_size=num_batch)
decoder = DecoderGRU(dec_inp_dim, 2, hidden_dim, z_dim, batch_size=num_batch)
model_gru = GRU_cVAE(encoder, decoder, opt_layer, inp_mean, inp_std).to(device)

P_ = torch.block_diag(P, P).to(device)
Pdot_ = Pdot.to(device)
Pddot_ = Pddot.to(device)

[autoreload of gru_cvae_aug_ddn failed: Traceback (most recent call last):
  File "/home/vikrant/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/vikrant/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/home/vikrant/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/home/vikrant/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 329, in update_class
    if update_generic(old_obj, new_obj):
  File "/home/vikrant/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/home/vikrant/.local/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 329, in update_class
    if update_generic(old_obj, new_obj):
  File "/home/vikrant/.

In [186]:
epochs = 60
step, beta = 0, 3.5
optimizer = optim.AdamW(model.parameters(), lr = 1e-5, weight_decay=6e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 12, gamma = 0.1)

avg_train_loss, avg_rcl_loss, avg_kl_loss, avg_aug_loss = [], [], [], []
for epoch in range(epochs):
    
    # Train Loop
    losses_train, kl_losses, rcl_losses, aug_losses = [], [], [], []
    model.train()
    from tqdm import tqdm
    it = 0
    for inp, out, past, future, inputs in tqdm(train_loader):
        it = it + 1
        inp = inp.to(device)
        out = out.to(device)
        
        past = past.to(device)
        future = future.to(device)
        b_inp = inputs.to(device) 

        traj_gt = out#torch.cat((out[:, :50], out[:, 50:]))
        # z = torch.cat([inp, out], dim = 1)
		# Sample from z -> Reparameterized 
		# z = self._sample_z(mean, std)
		
		# Decode y
		# y_star = self._decoder(z, inp_norm, init_state_ego, y_ub, y_lb)
        
        # Ego vehicle states
        initial_state_ego = inp[:, 2:6]
        initial_state_ego[:, 2] = inp[:, 2::5][:, -1]
        initial_state_ego[:, 3] = inp[:, 3::5][:, -1]
        initial_state_ego[:, 0:2] = 0
        
        # mean, std = model._encoder(inp, traj_gt)
        # z = model._sample_z(mean, std)
        # y_star = model._decoder(z, inp, initial_state_ego, 0, 0) 
        # traj_sol = (P @ y_star.T).T 
               
        # Remember to add the Aug Loss
        KL_loss, RCL_loss, loss, _ = model.forward(inp, traj_gt, initial_state_ego, P_, Pdot_, Pddot_, beta, step)
        KL_loss_, RCL_loss_, loss_, _ = model_gru.forward(inp, traj_gt, initial_state_ego, P_, Pdot_, Pddot_, beta, step)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses_train.append(loss.detach().cpu().numpy()) 
        rcl_losses.append(RCL_loss.detach().cpu().numpy())
        kl_losses.append(KL_loss.detach().cpu().numpy())
        # aug_losses.append(Aug.detach().cpu().numpy())
        if it % 10 == 1:    
            print(f"Epoch: {epoch + 1}, Train Loss: {np.average(losses_train):.3f}, RCL: {np.average(rcl_losses):.3f}, KL: {np.average(kl_losses):.3f}") #, Aug: {np.average(aug_losses):.3f}")

    if epoch % 1 == 0:
        print(f"Epoch: {epoch + 1}, Train Loss: {np.average(losses_train):.3f}, RCL: {np.average(rcl_losses):.3f}, KL: {np.average(kl_losses):.3f}") #, Aug: {np.average(aug_losses):.3f}")

    step += 1.0
    scheduler.step()
    avg_train_loss.append(np.average(losses_train)), avg_rcl_loss.append(np.average(rcl_losses)), \
    avg_kl_loss.append(np.average(kl_losses)) #, avg_aug_loss.append(np.average(aug_losses))

  0%|                                                                                                                   | 0/391 [00:00<?, ?it/s]

torch.Size([64, 2]) BHUVANIDAS
torch.Size([64, 202]) LALA
torch.Size([64, 2])
torch.Size([64, 202])
torch.Size([64, 1, 202]) torch.Size([1, 64, 2048]) OKOK


  0%|                                                                                                                   | 0/391 [00:10<?, ?it/s]


IndexError: index 2 is out of bounds for dimension 1 with size 2

In [77]:
# Sanity Check
for batch_num, (datas) in enumerate(train_loader):
    inp, out = datas
    
    print(inp.shape, out.shape)
        
    num = 2
    
    # # Sanity Check
    # x_obs = inp[num].flatten()[5::5]
    # y_obs = inp[num].flatten()[6::5]
     
    # th = np.linspace(0, 2 * np.pi, 100)
    plt.figure(1)
    
    a_obs, b_obs = 5.8, 3.2
    
    # for i in range(0, 10):
    #     x_ell = x_obs[i] + a_obs * np.cos(th)
    #     y_ell = y_obs[i] + b_obs * np.sin(th)
    #     plt.plot(x_ell, y_ell, '-k', linewidth=1.0)

    plt.axis('equal')
        
    traj_gt = out

    # Ego vehicle states
    initial_state_ego = inp[:, 2:6]
    initial_state_ego[:, 2] = inp[:, 2::5][:, -1]
    initial_state_ego[:, 3] = inp[:, 3::5][:, -1]
    initial_state_ego[:, 0:2] = 0

    mean, std = model._encoder(inp, traj_gt)
    z = model._sample_z(mean, std)
    y_star = model._decoder(z, inp, initial_state_ego, 0, 0) 
    traj_sol = (P @ y_star.T).T

    ccx = traj_sol[num].flatten()[0:60]
    ccy = traj_sol[num].flatten()[60:]
    
    cx = out[num].flatten()[0:60]
    cy = out[num].flatten()[60:]
    
    x_gt =  cx
    y_gt =  cy

    x_gt_ =  ccx
    y_gt_ =  ccy    
    
    x_obs = inp[0::5]
    y_obs = inp[1::5]
    
    ag_x = inp[num].flatten()[0::5]
    ag_y = inp[num].flatten()[1::5]    

    # plt.plot(ag_x.numpy(), ag_y.numpy(), label="Observed", color="blue")    
    plt.plot(x_gt.numpy(), y_gt.numpy(), label="Ground Truth", color="red")
    plt.plot(x_gt_.detach().numpy(), y_gt_.detach().numpy(), label="Predicted", color="orange")
    plt.legend()
    plt.savefig(f"argoverse_figs/{batch_num}_{num}.png")
    plt.clf()
    # plt.show()
    
    # break

RuntimeError: mat1 and mat2 shapes cannot be multiplied (4290x0 and 22x200)

In [10]:
torch.save(model.state_dict(), './Weights/cvae_aug_mse.pth')